<a href="https://colab.research.google.com/github/venuraja79/TfSamples/blob/master/Classification_on_structured_data_credit_card_fraud_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tf-nightly-gpu-2.0-preview

## First, vectorize the CSV data

In [0]:
import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/downloads/creditcardfraud.zip/
fname = '/Users/fchollet/Downloads/creditcard.csv'

all_features = []
all_targets = []
with open(fname) as f:
  for i, line in enumerate(f):
    if i == 0:
      print('HEADER:', line.strip())
      continue  # Skip header
    fields = line.strip().split(',')
    all_features.append([float(v.replace('"', '')) for v in fields[:-1]])
    all_targets.append([int(fields[-1].replace('"', ''))])
    if i == 1:
      print('EXAMPLE FEATURES:', all_features[-1])
    
features = np.array(all_features, dtype='float32')
targets = np.array(all_targets, dtype='uint8')
print('features.shape:', features.shape)
print('targets.shape:', targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


## Prepare a validation set

In [0]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

In [0]:
print('Number of training samples:', len(train_features))
print('Number of validation samples:', len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


## Analyze class imbalance in the targets

In [0]:
counts = np.bincount(train_targets[:, 0])
print('Number of positive samples in training data: {} ({:.2f}% of total)'.format(counts[1], 100 * float(counts[1]) / len(train_targets)))

Number of positive samples in training data: 417 (0.18% of total)


In [0]:
weight_for_0 = 1. / counts[0]
weight_for_1 = 1. / counts[1]

## Normalize the data using training set statistics

In [0]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

In [0]:
from tensorflow import keras

model = keras.Sequential([
  keras.layers.Dense(256, activation='relu',
                     input_shape=(train_features.shape[-1],)),
  keras.layers.Dense(256, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(256, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(1, activation='sigmoid'),
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               7936      
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 139,777
Trainable params: 139,777
Non-trainable params: 0
________________________________________________

In [0]:
metrics = [keras.metrics.FalseNegatives(name='fn'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.TrueNegatives(name='tn'),
           keras.metrics.TruePositives(name='tp'),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall')]

model.compile(optimizer=keras.optimizers.Adam(1e-2),
              loss='binary_crossentropy',
              metrics=metrics)

callbacks = [keras.callbacks.ModelCheckpoint('fraud_model_at_epoch_{epoch}.h5')]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(train_features, train_targets,
          batch_size=2048,
          epochs=50,
          verbose=2,
          callbacks=callbacks,
          validation_data=(val_features, val_targets),
          class_weight=class_weight)

Train on 227846 samples, validate on 56961 samples
Epoch 1/50
227846/227846 - 2s - loss: 2.4066e-06 - fn: 45.0000 - fp: 24340.0000 - tn: 203089.0000 - tp: 372.0000 - precision: 0.0151 - recall: 0.8921 - val_loss: 0.1311 - val_fn: 10.0000 - val_fp: 1103.0000 - val_tn: 55783.0000 - val_tp: 65.0000 - val_precision: 0.0557 - val_recall: 0.8667
Epoch 2/50
227846/227846 - 1s - loss: 1.4066e-06 - fn: 29.0000 - fp: 8700.0000 - tn: 218729.0000 - tp: 388.0000 - precision: 0.0427 - recall: 0.9305 - val_loss: 0.1044 - val_fn: 7.0000 - val_fp: 1521.0000 - val_tn: 55365.0000 - val_tp: 68.0000 - val_precision: 0.0428 - val_recall: 0.9067
Epoch 3/50
227846/227846 - 1s - loss: 1.1769e-06 - fn: 30.0000 - fp: 6944.0000 - tn: 220485.0000 - tp: 387.0000 - precision: 0.0528 - recall: 0.9281 - val_loss: 0.1048 - val_fn: 8.0000 - val_fp: 1291.0000 - val_tn: 55595.0000 - val_tp: 67.0000 - val_precision: 0.0493 - val_recall: 0.8933
Epoch 4/50
227846/227846 - 1s - loss: 1.1076e-06 - fn: 23.0000 - fp: 7688.0000 -

At the end of training, out of 56,961 validation transactions, we are:

- Correctly identifying 66 of them as fraudulent
- Missing 9 fraudulent transactions
- At the cost of incorrectly flagging 441 legitimate transactions

In the real world, one would put an even higher weight on class 1,
so as to reflect that False Negatives are more costly than False Positives.

Next time your credit card gets  declined in an online purchase -- this is why.